---

## Floating-point arithmetic

Real numbers are stored on a computer following the IEEE floating-point standard:

1. **half precision** using 16 bits (Julia type: `Float16`)
2. **single precision** using 32 bits (Julia type: `Float32`)
3. **double precision** using 64 bits (Julia type: `Float64`)

Julia also has an **arbitrary precision** floating-point data type called `BigFloat`. It is excellent if you need more precision, but it is also much slower.

In [1]:
?AbstractFloat

search: AbstractFloat



```
AbstractFloat <: Real
```

Abstract supertype for all floating point numbers.


In [2]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16
 Float32
 Float64

---

## Description of IEEE double floating-point format (`Float64`)

Suppose $x$ is a floating-point number stored in the following 64-bits:

$$
\begin{array}{|c|c|c|c|c|c|c|}
\hline
1 & 2 & \cdots & 12 & 13 & \cdots & 64 \\
\hline
s & e_{10} & \cdots & e_0 & f_1 & \cdots & f_{52} \\
\hline
\end{array}
$$

- 1 bit $s$ represents the **sign**
- 11 bits $e_{10} \cdots e_{0}$ represent the **exponent**
- 52 bits $f_1 \cdots f_{52}$ represent the **fraction** (a.k.a. the mantissa or significand)

Then

$$ x = (-1)^s \left[1.f_1 \cdots f_{52}\right]_2 \times 2^{(e-1023)}.$$

Notes: 

- $x$ is **normalized** to have its first digit nonzero.
- $e = \left[e_{10} \cdots e_{0}\right]_2 = e_{10} 2^{10} + \cdots + e_1 2^1 + e_0 2^0 \in \left[0, 2^{11}-1\right] = [0, 2047]$
- $e = 0$ and $e = 2047$ are reserved for special floating-point values, so 

$$e \in [1, 2046]$$

- the "$-1023$" in the exponent is called the **bias**:  $e-1023 \in [-1022,1023]$
- $\left[1.f_1 \cdots f_{52}\right]_2 = 1 + \frac{f_1}{2^1} + \frac{f_2}{2^2} + \cdots + \frac{f_{52}}{2^{52}}$


---

## Example

$$
\begin{split}
x & = -[1.101101]_2 \times 2^{(1026-1023)} \\
  & = -[1.101101]_2 \times 2^{3} \\
  & = -[1101.101]_2 \\
  & = -\left(1 \cdot 8 + 1 \cdot 4 + 0 \cdot 2 + 1 \cdot 1 + 1 \cdot \frac{1}{2} + 0 \cdot \frac{1}{4} + 1 \cdot \frac{1}{8}\right)  \\
  & = -13.625
\end{split}
$$

In [3]:
?bitstring

search: bitstring SubstitutionString



```
bitstring(n)
```

A string giving the literal bit representation of a number.

# Examples

```jldoctest
julia> bitstring(4)
"0000000000000000000000000000000000000000000000000000000000000100"

julia> bitstring(2.2)
"0100000000000001100110011001100110011001100110011001100110011010"
```


In [1]:
s = bitstring(-13.625)

"1100000000101011010000000000000000000000000000000000000000000000"

In [2]:
s[1], s[2:12], s[13:64]

('1', "10000000010", "1011010000000000000000000000000000000000000000000000")

In [3]:
Int(0b10000000010)

1026

In [4]:
function mybitstring(x::Float64)
    s = bitstring(x)
    return s[1], s[2:12], s[13:64]
end

mybitstring (generic function with 1 method)

In [5]:
mybitstring(-13.625)

('1', "10000000010", "1011010000000000000000000000000000000000000000000000")

In [7]:
methods(mybitstring)

# 1 method for generic function "mybitstring":
[1] mybitstring(x::Float64) in Main at In[4]:1

In [8]:
mybitstring(10)

LoadError: MethodError: no method matching mybitstring(::Int64)
Closest candidates are:
  mybitstring(!Matched::Float64) at In[4]:1

---

## Example

Even if a number can be represented exactly in base-10 with a finite number of digits, it may require an infinite number of digits in base-2.

$$
0.1 = \left[0.000110011001\ldots\right]_2 = \left[1.\overline{1001}\right]_2 \times 2^{-4}
$$

Therefore, $0.1$ cannot be represented exactly as a floating-point number.

In [14]:
# Show that 0.1 is not represented exactly as a Float64

x = 0.1
mybitstring(x)

('0', "01111111011", "1001100110011001100110011001100110011001100110011010")

In [15]:
y = BigFloat(x)

0.1000000000000000055511151231257827021181583404541015625

In [17]:
BigFloat("0.1")

0.1000000000000000000000000000000000000000000000000000000000000000000000000000002

In [18]:
BigFloat("0.1") - 0.1

-5.551115123125782702118158340454101562499999999999999999999999784095786122638884e-18

---

## Limits of floating-point numbers

- **Largest** `Float64` $= \left(2 - 2^{-52}\right) \times 2^{1023} \approx 2 \times 10^{308}$
- **Smallest positive normalized** `Float64` $= 2^{-1022} \approx 2 \times 10^{-308}$

In [19]:
?floatmax

search: floatmax floatmin



```
floatmax(T = Float64)
```

Return the largest finite number representable by the floating-point type `T`.

# Examples

```jldoctest
julia> floatmax(Float16)
Float16(6.55e4)

julia> floatmax(Float32)
3.4028235f38

julia> floatmax()
1.7976931348623157e308
```


In [31]:
# Experiment with floatmax
x = floatmax(Float64)

1.7976931348623157e308

In [21]:
mybitstring(x)

('0', "11111111110", "1111111111111111111111111111111111111111111111111111")

In [23]:
(x + 1.0) - x

0.0

In [24]:
10.0*x

Inf

In [25]:
(10.0*x)/10.0

Inf

In [32]:
-10.0*x

-Inf

---

In [26]:
?floatmin

search: floatmin floatmax



```
floatmin(T = Float64)
```

Return the smallest positive normal number representable by the floating-point type `T`.

# Examples

```jldoctest
julia> floatmin(Float16)
Float16(6.104e-5)

julia> floatmin(Float32)
1.1754944f-38

julia> floatmin()
2.2250738585072014e-308
```


In [33]:
# Experiment with floatmin
x = floatmin(Float64)

2.2250738585072014e-308

In [28]:
mybitstringing(x)

('0', "00000000001", "0000000000000000000000000000000000000000000000000000")

In [29]:
x/10.0

2.225073858507203e-309

In [30]:
mybitstring(x/10.0)

('0', "00000000000", "0001100110011001100110011001100110011001100110011010")

In [39]:
x/1e16

0.0

In [40]:
(x/1e16)*1e16

0.0

---

## De-normalized floating-point numbers

The IEEE floating-point standard also allows **de-normalized** numbers that are smaller than `floatmin(Float64)`. De-normalized floats are represented by $e = 0$.

In [44]:
# Compute the smallest Float64 that is not zero

x = 1.0
n = 0
while x != 0.0
    x /= 2.0
    n -= 1
end

x, n

(0.0, -1075)

In [45]:
2.0^-1075

0.0

In [47]:
x = 2.0^-1074

5.0e-324

In [48]:
mybitstring(x)

('0', "00000000000", "0000000000000000000000000000000000000000000000000001")

In [50]:
(x/2.0)*2.0 == x

false

---

## Other special floats

- `0.0` and `-0.0`: $$e_{10} \cdots e_0 = 0 \cdots 0 \quad \text{and} \quad f_1 \cdots f_{52} = 0 \cdots 0$$
- `Inf` and `-Inf`: $$e_{10} \cdots e_0 = 1 \cdots 1 \quad \text{and} \quad f_1 \cdots f_{52} = 0 \cdots 0$$
- `NaN` (not-a-number): $$e_{10} \cdots e_0 = 1 \cdots 1 \quad \text{and} \quad f_1 \cdots f_{52} \neq 0$$

From [Julia Manual: Mathematical Operations and Elementary Functions](https://docs.julialang.org/en/v1/manual/mathematical-operations/):

- `Inf` is equal to itself and greater than everything else except `NaN`.
- `-Inf` is equal to itself and less then everything else except `NaN`.
- `NaN` is not equal to, not less than, and not greater than anything, including itself.

In [51]:
# Experiment with 0.0, -0.0, Inf, -Inf, and NaN

mybitstring(0.0)

('0', "00000000000", "0000000000000000000000000000000000000000000000000000")

In [52]:
mybitstring(-0.0)

('1', "00000000000", "0000000000000000000000000000000000000000000000000000")

In [53]:
Inf - Inf

NaN

In [54]:
0.0*Inf

NaN

In [55]:
1.0/0.0

Inf

In [56]:
mybitstring(NaN)

('0', "11111111111", "1000000000000000000000000000000000000000000000000000")

In [57]:
-0.0 < 0.0

false

In [58]:
-0.0 == 0.0

true

In [59]:
-0.0 === 0.0

false

In [60]:
?===

search: === == !==



```
===(x,y) -> Bool
≡(x,y) -> Bool
```

Determine whether `x` and `y` are identical, in the sense that no program could distinguish them. First the types of `x` and `y` are compared. If those are identical, mutable objects are compared by address in memory and immutable objects (such as numbers) are compared by contents at the bit level. This function is sometimes called "egal". It always returns a `Bool` value.

# Examples

```jldoctest
julia> a = [1, 2]; b = [1, 2];

julia> a == b
true

julia> a === b
false

julia> a === a
true
```


---

## Machine epsilon `eps(Float64)` and the unit roundoff $\eta$

- `1.0 + eps(Float64)` is the first `Float64` that is larger than `1.0`

$$\mathtt{eps(Float64)} = 2^{-52} \approx 2.2 \times 10^{-16}$$

- $\eta = $ `eps(Float64)/2.0` is the largest possible **relative error** due to roundoff

$$\eta = 2^{-53} \approx 1.1 \times 10^{-16}$$

In [61]:
?eps

search: eps @elapsed expanduser supertypes escape_string setprecision peakflops



```
eps(::Type{T}) where T<:AbstractFloat
eps()
```

Return the *machine epsilon* of the floating point type `T` (`T = Float64` by default). This is defined as the gap between 1 and the next largest value representable by `typeof(one(T))`, and is equivalent to `eps(one(T))`.  (Since `eps(T)` is a bound on the *relative error* of `T`, it is a "dimensionless" quantity like [`one`](@ref).)

# Examples

```jldoctest
julia> eps()
2.220446049250313e-16

julia> eps(Float32)
1.1920929f-7

julia> 1.0 + eps()
1.0000000000000002

julia> 1.0 + eps()/2
1.0
```

---

```
eps(x::AbstractFloat)
```

Return the *unit in last place* (ulp) of `x`. This is the distance between consecutive representable floating point values at `x`. In most cases, if the distance on either side of `x` is different, then the larger of the two is taken, that is

```
eps(x) == max(x-prevfloat(x), nextfloat(x)-x)
```

The exceptions to this rule are the smallest and largest finite values (e.g. `nextfloat(-Inf)` and `prevfloat(Inf)` for [`Float64`](@ref)), which round to the smaller of the values.

The rationale for this behavior is that `eps` bounds the floating point rounding error. Under the default `RoundNearest` rounding mode, if $y$ is a real number and $x$ is the nearest floating point number to $y$, then

$$
|y-x| \leq \operatorname{eps}(x)/2.
$$

# Examples

```jldoctest
julia> eps(1.0)
2.220446049250313e-16

julia> eps(prevfloat(2.0))
2.220446049250313e-16

julia> eps(2.0)
4.440892098500626e-16

julia> x = prevfloat(Inf)      # largest finite Float64
1.7976931348623157e308

julia> x + eps(x)/2            # rounds up
Inf

julia> x + prevfloat(eps(x)/2) # rounds down
1.7976931348623157e308
```

---

```
eps(::Type{DateTime}) -> Millisecond
eps(::Type{Date}) -> Day
eps(::Type{Time}) -> Nanosecond
eps(::TimeType) -> Period
```

Return the smallest unit value supported by the `TimeType`.

# Examples

```jldoctest
julia> eps(DateTime)
1 millisecond

julia> eps(Date)
1 day

julia> eps(Time)
1 nanosecond
```


In [62]:
# Experiment with eps
ϵ = eps()

2.220446049250313e-16

In [65]:
η = ϵ/2.0

1.1102230246251565e-16

In [63]:
1.0 + ϵ

1.0000000000000002

In [64]:
(1.0 + ϵ) - 1.0

2.220446049250313e-16

In [66]:
1.0 + η

1.0

In [67]:
(1.0 + η) - 1.0

0.0

In [68]:
?nextfloat

search: nextfloat



```
nextfloat(x::AbstractFloat, n::Integer)
```

The result of `n` iterative applications of `nextfloat` to `x` if `n >= 0`, or `-n` applications of `prevfloat` if `n < 0`.

---

```
nextfloat(x::AbstractFloat)
```

Return the smallest floating point number `y` of the same type as `x` such `x < y`. If no such `y` exists (e.g. if `x` is `Inf` or `NaN`), then return `x`.


In [69]:
?prevfloat

search: prevfloat



```
prevfloat(x::AbstractFloat, n::Integer)
```

The result of `n` iterative applications of `prevfloat` to `x` if `n >= 0`, or `-n` applications of `nextfloat` if `n < 0`.

---

```
prevfloat(x::AbstractFloat)
```

Return the largest floating point number `y` of the same type as `x` such `y < x`. If no such `y` exists (e.g. if `x` is `-Inf` or `NaN`), then return `x`.


In [71]:
mybitstring(1.0)

('0', "01111111111", "0000000000000000000000000000000000000000000000000000")

In [72]:
mybitstring(1.0 + eps())

('0', "01111111111", "0000000000000000000000000000000000000000000000000001")

In [73]:
nextfloat(1.0) - 1.0

2.220446049250313e-16

In [74]:
2.0 - prevfloat(2.0)

2.220446049250313e-16

In [75]:
nextfloat(2.0) - 2.0

4.440892098500626e-16

In [76]:
x = 2.0^50.0
nextfloat(x) - x

0.25

In [77]:
x = 2.0^51.0
nextfloat(x) - x

0.5

In [78]:
x = 2.0^52.0
nextfloat(x) - x

1.0

In [79]:
x = 2.0^53.0
nextfloat(x) - x

2.0

---

## Roundoff error example

Suppose we are using a base-10 floating-point system with 4 significant digits, using `RoundNearest`:

$$
\begin{split}
\left( 1.112 \times 10^1 \right) \times \left( 1.112 \times 10^2 \right)
& = 1.236544 \times 10^3 \\
& \rightarrow 1.237 \times 10^3 = 1237
\end{split}
$$

The absolute error is $1237 - 1236.544 = 0.456$.

The relative error is $$\frac{0.456}{1236.544} \approx 0.0004 = 0.04 \%$$

The default rounding mode is `RoundNearest` (round to the nearest floating-point number). This implies that

$$ \frac{|x - \mathrm{fl}(x)|}{|x|} \leq \eta.$$

If `RoundToZero` is used (a.k.a. **chopping**), then

$$ \frac{|x - \mathrm{fl}(x)|}{|x|} \leq 2 \eta.$$

`RoundNearest` is used since it produces smaller roundoff errors.

---

## Roundoff error accumulation

When performing arithmetic operations on floats, extra **guard digits** are used to ensure **exact rounding**. This guarantees that the relative error of a floating-point operation (**flop**) is small. More precisely, for floating-point numbers $x$ and $y$, we have

$$
\begin{split}
\mathrm{fl}(x \pm y) &= (x \pm y)(1 + \varepsilon_1) \\
\mathrm{fl}(x \times y) &= (x \times y)(1 + \varepsilon_2) \\
\mathrm{fl}(x \div y) &= (x \div y)(1 + \varepsilon_3) \\
\end{split}
$$

where $|\varepsilon_i| \leq \eta$, for $i = 1,2,3$, where $\eta$ is the unit roundoff.

Although the relative error of each flop is small, it is possible to have the roundoff error accumulate and create significant error in the final result. If $E_n$ is the error after $n$ flops, then:

- **linear roundoff error accumulation** is when $E_n \approx c_0 n E_0$
- **exponential roundoff error accumulation** is when $E_n \approx c_1^n E_0$, for some $c_1 > 1$

In general, linear roundoff error accumulation is unavoidable. On the other hand, exponential roundoff error accumulation is not acceptable and is an indication of an **unstable algorithm**. (See Example 1.6 in Ascher-Greif for an example of exponential roundoff error accumulation, and see Exercise 5 in Section 1.4 for a numerically stable method to accomplish the same task.)

---

## General advice

1. Adding $x + y$ when $|x| \gg |y|$ can cause the information in $y$ to be 'lost' in the summation.

2. Dividing by very small numbers or multiplying by very large numbers can **magnify error**.

3. Subtracting numbers that are almost equal produces **cancellation error**.

4. An **overflow** occurs when the result is too large in magnitude to be representable as a float. Result will become either `Inf` or `-Inf`. Overflows should be avoided.

4. An **underflow** occurs when the result is too small in magnitude to be representable as a float. Result will become either `0.0` or `-0.0`.


---

## Example (summation order)

This next example shows that summation order can make a difference. We will compute

$$
s = \sum_{n = 1}^{1,000,000} \frac{1}{n}
$$

in two different ways: from largest to smallest and from smallest to largest.

In [ ]:
# Sum from largest to smallest


In [ ]:
# Sum from smallest to largest


---

## Example (cancellation error)

Show that 

$$
\ln\left( x - \sqrt{x^2-1} \right) = -\ln\left( x + \sqrt{x^2-1} \right).
$$

Which formula is more suitable for numerical computation?

In [ ]:
# Experiment with both formulas


---

## Example (avoiding overflow)

Overflow is possible when squaring a large number. This needs to be avoided when computing the Euclidean norm (a.k.a. the $2$-norm) of a vector $x$:

$$
\|x\|_2 = \sqrt{x_1^2 + x_2^2 + \cdots + x_n^2}.
$$

If some $x_i$ is very large, it is possible that $x_i^2$ will overflow, causing the final result to be `Inf`. We can avoid this as follows.

Let 
$$\bar{x} = \max_{i=1:n} |x_i|.$$
Then
$$
\|x\|_2 = \bar{x} \sqrt{\left(\frac{x_1}{\bar{x}}\right)^2 + \left(\frac{x_2}{\bar{x}}\right)^2 + \cdots + \left(\frac{x_n}{\bar{x}}\right)^2}.
$$
Since $|x_i/\bar{x}| \leq 1$ for all $i$, no overflow will occur. Underflow may occur, but this is harmless.


In [ ]:
# Experiment with both formulas
n = 100

x = randn(n)
x[1] = 1e200

using LinearAlgebra
norm(x)

---